# Load Libraries :

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.ndimage import gaussian_filter, sobel

# 1 - Fundamental Level Sets (1980s-1990s)

## Introduction 
Instead of representing the active contour explicitly by a set of ordered point, We define the contour implicitely by the zero level set of a function phi(x,y), by evolving this function in time we can approch our desired contour.  
During evolution:
- *ϕ* is updated over time using a **partial differential equation (PDE)**.
- The idea is that *ϕ* deforms under some speed function so that the zero-level set (the contour) moves toward the object boundary in the image.

## Mathematical Formulation

The basic level set method represents a contour 𝐶 implicitly as the zero level set of a higher-dimensional function ϕ(x,y,t), where:
- (x,y) are spatial coordinates
- t is time/iteration
- The contour C is defined where ϕ(x,y,t) = 0

The contour evolution is governed by the level set equation:

$$\frac{\partial \phi}{\partial t} + F|\nabla \phi| = 0$$

Where:
- F is the speed function that controls the contour evolution
- ∇ϕ is the spatial gradient of ϕ

## Basic Algorithm Components

### 1. Level Set Function Initialization

Typically, the level set function is initialized as a signed distance function:
- ϕ(x,y,0) < 0 for points inside the initial contour
- ϕ(x,y,0) > 0 for points outside
- |ϕ(x,y,0)| represents the distance to the contour

A common initialization is:
$$\phi(x,y,0) = \pm d$$

Where d is the distance from (x,y) to the initial contour, and the sign depends on whether the point is inside (negative) or outside (positive).

### 2. Speed Function Design

For image segmentation, the speed function F typically depends on:
- Image gradient: to stop the contour at edges
- Curvature: to maintain smoothness

A basic speed function:
$$ F = g \cdot \kappa + \nabla g \cdot \nabla \phi $$

Where:
- g is an edge detector function (e.g., g = 1/(1+|∇I|²))
- I is the image
- κ is the curvature of the level set
- c (optional) is a constant (balloon force)

### 3. Level Set Evolution

At each iteration, update ϕ using:
$$\phi^{n+1} = \phi^n - \Delta t \cdot F|\nabla \phi^n|$$


#### Time Step Selection

For stability, the time step must satisfy the CFL condition:
$$\Delta t \leq \frac{\Delta x}{max(|F|)}$$

## Complete Algorithm

```
1. Initialize φ(x,y,0) as a signed distance function
2. Calculate the edge indicator function g(|∇I|)
3. For each iteration n:
   a. Calculate ∇φⁿ using finite differences
   b. Compute curvature κ
   c. Evaluate speed function F
   d. Update φⁿ⁺¹ = φⁿ - Δt·F|∇φⁿ|
   e. Periodically reinitialize φ to maintain signed distance property
4. Extract final contour from zero level set φ = 0
```



## Implementation :

In [ ]:
# Load image
img = cv2.imread("coin.png", cv2.IMREAD_GRAYSCALE)

plt.figure()
plt.imshow(img, cmap='gray')
plt.title('Input Image')
plt.axis('off')
plt.show()

### 1. Initialize the LSF : φ(x,y,0)

In [ ]:
# 1. Initialize the LSF : φ(x,y,0)
def initialize_level_set_circle(shape, center=None, radius=None):
    """
    Initializes a level set function phi as a signed distance function to a circle.

    Parameters:
    - shape: tuple of ints, (height, width) of the domain
    - center: tuple (cx, cy) for the center of the circle; defaults to image center
    - radius: int, radius of the initial circle

    Returns:
    - phi: 2D numpy array of shape `shape`, representing the level set function
    """
    h, w = shape
    if center is None:
        center = (w // 2, h // 2)

    if radius is None:
        radius = (h // 2) - (h // 50)

    x = np.arange(0, w)
    y = np.arange(0, h)
    X, Y = np.meshgrid(x, y)

    # Signed distance to the circle: negative inside, zero on boundary, positive outside
    phi_init = np.sqrt((X - center[0])**2 + (Y - center[1])**2) - radius

    return phi_init

In [ ]:
# initialise the LSF as a signed distance function ( |∇φ| = 1 ) :

phi_init = initialize_level_set_circle(img.shape)

plt.figure()
plt.imshow(phi_init, cmap='coolwarm')
plt.colorbar()
plt.contour(phi_init, levels=[0], colors='r')  # zero level set
plt.title('Initial LSF')
plt.axis('off')
plt.show()

### 2. Calculate the edge indicator function g(|∇I|)
$$ g(|\nabla I|) = \frac{1}{1 + |\nabla I|^2} $$  
This function is **small near strong edges**, causing the evolution to slow down and stop there

In [ ]:
# 2. Calculate the edge indicator function g(|∇I|)
def compute_edge_indicator(img, sigma=1.0):
    """
    Compute the edge indicator function g for level set evolution.
    
    Parameters:
    - img: 2D array, input grayscale image
    - sigma: float, standard deviation for Gaussian smoothing
    
    Returns:
    - g: 2D array, edge indicator function
    """
    # Convert image to float if not already
    img = img.astype(float)
    
    # Normalize image to [0,1] if needed
    if np.max(img) > 1.0:
        img = img / 255.0
    
    # Step 1: Apply Gaussian smoothing to reduce noise
    img_smooth = gaussian_filter(img, sigma)
    
    # Step 2: Compute gradients using Sobel operators
    grad_x = sobel(img_smooth, axis=1)
    grad_y = sobel(img_smooth, axis=0)
    
    # Step 3: Compute gradient magnitude
    grad_mag = np.sqrt(grad_x**2 + grad_y**2 + 1e-10)  # Small epsilon to avoid division by zero
    
    # Step 4: Compute edge indicator function (standard formulation)
    g = 1.0 / (1.0 + grad_mag**2)
    
    # Apply a light smoothing to the edge indicator to reduce artifacts
    g = gaussian_filter(g, sigma=0.5)
    
    return g

In [ ]:
# compute the edge indicator function :

g = compute_edge_indicator(img)

plt.figure()
plt.imshow(g, cmap='gray')
plt.colorbar()
plt.title('edge indicator function g(|∇I|)')
plt.axis('off')
plt.show()

### 3. For each iteration n:
####    a. Calculate ∇φⁿ using finite differences  
- phi_x, phi_y for directional updates and geometric flow
- |∇φ| for evolving the zero level set in the normal direction

In [ ]:
# 3. For each iteration n:
#    a. Calculate ∇φⁿ using finite differences
def compute_phi_gradient(phi):
    """
    Compute the gradient of the level set function φ.

    Parameters:
    - phi: 2D array, the level set function

    Returns:
    - phi_x: derivative of φ in x direction (axis=1, columns)
    - phi_y: derivative of φ in y direction (axis=0, rows)
    - grad_phi_mag: gradient magnitude sqrt(φ_x^2 + φ_y^2)
    """
    grad_phi_y, grad_phi_x = np.gradient(phi)  # order: rows (y), columns (x)
    grad_phi_mag = np.sqrt(grad_phi_x**2 + grad_phi_y**2) + 1e-10  # small epsilon to avoid division by 0
    return grad_phi_x, grad_phi_y, grad_phi_mag

In [ ]:
# computing the gradients of phi :

grad_phi_x, grad_phi_y, grad_phi_mag = compute_phi_gradient(phi_init)

plt.figure(figsize=(15,5))

plt.subplot(1,3,3)
plt.imshow(grad_phi_mag, cmap='coolwarm', vmin=0, vmax=2)
plt.colorbar()
plt.title('grad_phi_mag')

plt.tight_layout()
plt.show()

####    b. Compute curvature κ  

$$ \kappa = \nabla \cdot \left( \frac{\nabla \phi}{|\nabla \phi|} \right) $$  
This curvature term is useful for:
- **Smoothing** the contour
- Regularizing its shape during evolution

In [ ]:
def compute_curvature(phi):
    """
    Compute the curvature of the level set function phi.
    
    The curvature is defined as the divergence of the normalized gradient:
    κ = div(∇φ/|∇φ|)
    
    Parameters:
    - phi: 2D array, level set function
    
    Returns:
    - curvature: 2D array, curvature of the level set
    """
    # Compute the gradient of phi
    phi_x, phi_y, grad_phi_mag = compute_phi_gradient(phi)
    
    # Compute the second derivatives
    # For x direction
    phi_xx = np.zeros_like(phi)
    phi_xx[:, 1:-1] = (phi[:, 2:] - 2*phi[:, 1:-1] + phi[:, :-2])  # Central difference for second derivative
    
    # For y direction
    phi_yy = np.zeros_like(phi)
    phi_yy[1:-1, :] = (phi[2:, :] - 2*phi[1:-1, :] + phi[:-2, :])  # Central difference for second derivative
    
    # Mixed derivative (xy)
    phi_xy = np.zeros_like(phi)
    phi_xy[1:-1, 1:-1] = (phi[2:, 2:] - phi[2:, :-2] - phi[:-2, 2:] + phi[:-2, :-2]) / 4.0
    
    # Small epsilon to avoid division by zero
    epsilon = 1e-10
    
    # Compute curvature using the formula:
    # κ = (φxx*φy² - 2*φxy*φx*φy + φyy*φx²) / (|∇φ|³)
    numerator = phi_xx * phi_y**2 - 2 * phi_xy * phi_x * phi_y + phi_yy * phi_x**2
    denominator = (grad_phi_mag**3 + epsilon)
    
    curvature = numerator / denominator
    
    # Limit the curvature values to avoid instabilities
    curvature = np.clip(curvature, -1.0, 1.0)
    
    return curvature

In [ ]:
# test the curvature function :

curvature = compute_curvature(phi_init)

plt.figure()
plt.imshow(curvature, cmap='gray')
plt.colorbar()
plt.title('curvature')
plt.axis('off')
plt.show()

#### c. Evaluate speed function F  

$$ F = g \cdot \kappa + \nabla g \cdot \nabla \phi $$
This form is equivalent to:
$$ F = g \cdot \kappa + g_x \cdot \phi_x + g_y \cdot \phi_y $$

In [ ]:
def compute_speed_function(phi, g, alpha=0.1, beta=1.2):
    """
    Compute the speed function for level set evolution.
    
    This implements a typical speed function that combines:
    1. Curvature-based regularization term
    2. Constant balloon force term
    3. External edge-based force term
    
    Parameters:
    - phi: 2D array, current level set function
    - g: 2D array, edge indicator function
    - alpha: float, weight of the balloon force term
    - beta: float, weight of the edge attraction term
    
    Returns:
    - speed: 2D array, speed function for level set evolution
    """
    # Compute curvature
    curvature = compute_curvature(phi)
    
    # Compute gradient of phi
    phi_x, phi_y, grad_phi_mag = compute_phi_gradient(phi)
    
    # Compute gradient of edge indicator function g
    g_x = np.zeros_like(g)
    g_y = np.zeros_like(g)
    
    # Central differences for interior points
    g_x[:, 1:-1] = (g[:, 2:] - g[:, :-2]) / 2.0
    g_y[1:-1, :] = (g[2:, :] - g[:-2, :]) / 2.0
    
    # Dot product of ∇g and ∇φ/|∇φ|
    epsilon = 1e-10  # Avoid division by zero
    nx = phi_x / (grad_phi_mag + epsilon)  # Normal vector x-component
    ny = phi_y / (grad_phi_mag + epsilon)  # Normal vector y-component
    edge_attraction = g_x * nx + g_y * ny
    
    # Compute speed function components:
    # 1. Curvature term: g * κ (regularization)
    # 2. Balloon force: g * α (expansion/contraction)
    # 3. Edge attraction: β * edge_attraction (pulls toward edges)
    speed = g * (curvature + alpha) + beta * edge_attraction
    
    return speed

In [ ]:
# compude the speed function used to update phi :

speed_function = compute_speed_function(phi_init, g, alpha=0.1, beta=1.2)

plt.figure()
plt.imshow(speed_function, cmap='coolwarm', vmin=-1, vmax=1)
plt.colorbar()
plt.title('Speed Function')
plt.axis('off')
plt.show()

#### d. Update φⁿ⁺¹ = φⁿ - Δt·F|∇φⁿ|
$$ \phi^{t+1} = \phi^t + \Delta t \cdot F |\nabla \phi| $$

#### e. Run iterative loop

In [ ]:
def level_set_contour_detection(image, nb_iter, center_initial=None, radius_initial=None, time_step=0.1, alpha=0.1, beta=1.2):
    """
    Perform level set evolution for contour detection.
    
    Parameters:
    - image: 2D array, grayscale image
    - nb_iter: int, number of iterations
    - center_initial: tuple, (x, y) coordinates of initial circle center
    - radius_initial: float, radius of initial circle
    - time_step: float, time step Δt
    - alpha: float, weight of the balloon force term
    - beta: float, weight of the edge attraction term
    
    Returns:
    - phi_n: 2D array, final level set function
    """

    iter_display = nb_iter//10

    # Initialize the LSF
    phi_init = initialize_level_set_circle(image.shape, center_initial, radius_initial)
    
    # Edge indicator function
    g = compute_edge_indicator(image)
    
    # Display inputs
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(image, cmap='gray')
    plt.contour(phi_init, levels=[0], colors='r')
    plt.title('Input Image and Initial LSF Contour')
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(g, cmap='gray')
    plt.title('Edge Indicator Function')
    plt.colorbar()
    plt.axis('off')
    
    plt.suptitle("Initial LSF and Input")
    plt.tight_layout()
    plt.show()
    
    # Initialize the loop
    phi_n = phi_init.copy()
    
    for i in range(nb_iter):
        # Compute ∇φ and |∇φ|
        phi_x, phi_y, grad_phi_mag = compute_phi_gradient(phi_n)
        
        # Compute speed function
        speed_function = compute_speed_function(phi_n, g, alpha=alpha, beta=beta)
        
        # Update φ
        phi_n = phi_n + time_step * speed_function * grad_phi_mag
        
        # Display every 10 iterations
        if ((i+1) % iter_display) == 0 :
            plt.figure(figsize=(20, 4))

            plt.subplot(1, 4, 1)
            plt.imshow(image, cmap='gray')
            plt.contour(phi_n, levels=[0], colors='r')
            plt.title('iteration Contour')
            plt.axis('off')
            
            plt.subplot(1, 4, 2)
            plt.imshow(speed_function, cmap='coolwarm', vmin=-1, vmax=1)
            plt.colorbar()
            plt.title('Speed Function')
            plt.axis('off')

            plt.subplot(1, 4, 3)
            plt.imshow(grad_phi_mag, cmap='coolwarm', vmin=0, vmax=2)
            plt.colorbar()
            plt.title('grad_phi_mag')
            plt.axis('off')
            
            plt.subplot(1, 4, 4)
            plt.imshow(phi_n, cmap='coolwarm', vmin=-1, vmax=1)
            plt.colorbar()
            plt.title('Level Set Function')
            plt.axis('off')
            
            plt.suptitle(f"Iteration {i+1}")
            plt.tight_layout()
            plt.show()
    
    # Final display
    plt.figure(figsize=(20, 4))

    plt.subplot(1, 4, 1)
    plt.imshow(image, cmap='gray')
    plt.contour(phi_n, levels=[0], colors='r')
    plt.title('iteration Contour')
    plt.axis('off')
    
    plt.subplot(1, 4, 2)
    plt.imshow(speed_function, cmap='coolwarm', vmin=-1, vmax=1)
    plt.colorbar()
    plt.title('Speed Function')
    plt.axis('off')

    plt.subplot(1, 4, 3)
    plt.imshow(grad_phi_mag, cmap='coolwarm', vmin=0, vmax=2)
    plt.colorbar()
    plt.title('grad_phi_mag')
    plt.axis('off')
    
    plt.subplot(1, 4, 4)
    plt.imshow(phi_n, cmap='coolwarm', vmin=-1, vmax=1)
    plt.colorbar()
    plt.title('Level Set Function')
    plt.axis('off')
    
    plt.suptitle("Final Display")
    plt.tight_layout()
    plt.show()
    
    return phi_n

In [ ]:
# test of the fundamental level set algorithm time_step=0.1, alpha=0.1, beta=1.2 :

img = cv2.imread("coins.png", cv2.IMREAD_GRAYSCALE)
phi_n = level_set_contour_detection(img, nb_iter=4000, time_step=0.1, alpha=0.1, beta=1.2)

## Conclusion:

- To prevent the evolution from diverging into artifacts across the level set function (LSF), the time step must be kept very small. However, this leads to significant computational cost.

- For the contour to evolve smoothly and accurately converge toward the edges, careful analysis and fine-tuning are required. In particular, selecting appropriate values for **α** (the weight of the balloon force that pushes the contour inward) and **β** (the weight of the edge attraction term that halts the evolution at boundaries) is highly dependent on the specific image.

- As iterations progress, the gradient magnitude |grad_phi| becomes increasingly steep, causing instabilities. This results in the formation of anomalies within the LSF, as well as irregular, unwanted contours near some inner edges.

##### For the algorithm to work the time_step must be small enough, as a result the level basic level sets algorithms with or without the reinitialisation are very computationally expensive.

In [ ]:
# test of the level set algorithm with time_step equal to 0.5 instead of 0.1 :

img = cv2.imread("coins.png", cv2.IMREAD_GRAYSCALE)
phi_n = level_set_contour_detection(img, 4000, time_step=0.5, alpha=0.1, beta=1.2)

# 2 - Fundamental Level Set with Reinitialization (1990s)

## Issue :  

As the level set evolves over iterations, it gradually loses its signed distance property ( |∇φ| = 1 ) due to numerical errors and the nature of the evolution equation. The level set function can develop steep or flat regions where |∇φ| is far from 1. This causes:

- Regions where |∇φ| >> 1: very sttep gradient : Numerical instability and excessive contour movement
- Regions where |∇φ| << 1: Slow or stalled contour evolution
- Regions where |∇φ| = 1: regular signed distance : The absolute value of φ at any point equals the shortest distance to the contour.  
  
In level set methods, we ideally want the level set function φ to maintain a signed distance property, meaning **|∇φ| = 1** everywhere. This property is important because:

1. **Numerical stability**: When φ is a signed distance function, gradients are well-behaved and numerical calculations are more stable.

2. **Improved accuracy**: The contour evolution equations are derived assuming φ is a signed distance function.

3. **Consistent evolution speed**: Without maintaining the distance property, some regions might evolve faster than others, leading to uneven contour movement.

## Solution :

We periodically **reinitialize** phi to be a signed distance function again, **while preserving the zero level set**, It is like reshaping the level set function while preserving the zero level set (contour).

### Mathematical Formulation

Reinitialization solves the following PDE to steady state:

$$\frac{\partial \phi}{\partial \tau} = \text{sign}(\phi_0)(1-|\nabla \phi|)$$

Where:
- τ is an artificial time parameter (not the same as the evolution time t)
- φ₀ is the level set function before reinitialization
- sign(φ₀) ensures we preserve the sign (inside/outside) information

This equation acts as follows:
- When |∇φ| > 1: The term (1-|∇φ|) is negative, so φ decreases
- When |∇φ| < 1: The term (1-|∇φ|) is positive, so φ increases
- When |∇φ| = 1: The term (1-|∇φ|) is zero, so φ remains unchanged

The sign function ensures that:
- Inside regions (φ₀ < 0) stay negative
- Outside regions (φ₀ > 0) stay positive
- The zero level set (φ₀ = 0) remains fixed

In [ ]:
def reinitialize_phi(phi, iterations=10, dt=0.1):
    """
    Reinitialize phi to be a signed distance function.
    
    Parameters:
    - phi: 2D array, level set function to reinitialize
    - iterations: int, number of iterations for reinitialization
    - dt: float, time step for reinitialization
    
    Returns:
    - phi_reinit: 2D array, reinitialized level set function
    """
    # Store the original sign of phi
    phi_0 = phi.copy()
    sign_phi_0 = np.sign(phi_0)
    
    # Smoothed sign function to avoid discontinuities
    def smoothed_sign(x, epsilon=1.5):
        return x / np.sqrt(x**2 + epsilon**2)
    
    # Initialize phi_reinit
    phi_reinit = phi.copy()
    
    # Iterate to solve the reinitialization equation
    for _ in range(iterations):
        # Compute upwind gradients for reinitialization
        phi_x_p = np.zeros_like(phi_reinit)
        phi_x_m = np.zeros_like(phi_reinit)
        phi_y_p = np.zeros_like(phi_reinit)
        phi_y_m = np.zeros_like(phi_reinit)
        
        # Forward differences
        phi_x_p[:, :-1] = phi_reinit[:, 1:] - phi_reinit[:, :-1]
        phi_y_p[:-1, :] = phi_reinit[1:, :] - phi_reinit[:-1, :]
        
        # Backward differences
        phi_x_m[:, 1:] = phi_reinit[:, 1:] - phi_reinit[:, :-1]
        phi_y_m[1:, :] = phi_reinit[1:, :] - phi_reinit[:-1, :]
        
        # Choose upwind differences based on sign
        a = smoothed_sign(sign_phi_0)
        
        # For a > 0, use backward differences for "+" terms and forward for "-" terms
        phi_x_2 = np.maximum(np.maximum(phi_x_m, 0)**2, np.minimum(phi_x_p, 0)**2)
        phi_y_2 = np.maximum(np.maximum(phi_y_m, 0)**2, np.minimum(phi_y_p, 0)**2)
        
        # For a < 0, use forward differences for "+" terms and backward for "-" terms
        phi_x_2[a < 0] = np.maximum(np.minimum(phi_x_m[a < 0], 0)**2, np.maximum(phi_x_p[a < 0], 0)**2)
        phi_y_2[a < 0] = np.maximum(np.minimum(phi_y_m[a < 0], 0)**2, np.maximum(phi_y_p[a < 0], 0)**2)
        
        # Compute gradient magnitude
        grad_phi_mag = np.sqrt(phi_x_2 + phi_y_2)
        
        # Update phi_reinit
        phi_reinit = phi_reinit - dt * sign_phi_0 * (grad_phi_mag - 1.0)
    
    return phi_reinit

In [ ]:
def level_set_contour_detection_fundamental_reinitialization(image, nb_iter, center_initial=None, radius_initial=None, time_step=0.1, alpha=0.1, beta=1.2, reinit_every = None, initialisation_iter = None, initialisation_time_step=None):
    """
    Perform level set evolution for contour detection.
    
    Parameters:
    - image: 2D array, grayscale image
    - nb_iter: int, number of iterations
    - center_initial: tuple, (x, y) coordinates of initial circle center
    - radius_initial: float, radius of initial circle
    - time_step: float, time step Δt
    - alpha: float, weight of the balloon force term
    - beta: float, weight of the edge attraction term
    
    Returns:
    - phi_n: 2D array, final level set function
    """

    # inputs
    if reinit_every == None :
        reinit_every = nb_iter//5

    iter_display = nb_iter//10

    # Initialize the LSF
    phi_init = initialize_level_set_circle(image.shape, center_initial, radius_initial)
    
    # Edge indicator function
    g = compute_edge_indicator(image)
    
    # Display inputs
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(image, cmap='gray')
    plt.contour(phi_init, levels=[0], colors='r')
    plt.title('Input Image and Initial LSF Contour')
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(g, cmap='gray')
    plt.title('Edge Indicator Function')
    plt.colorbar()
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Initialize the loop
    phi_n = phi_init.copy()
    
    for i in range(nb_iter):
        # Compute ∇φ and |∇φ|
        phi_x, phi_y, grad_phi_mag = compute_phi_gradient(phi_n)
        
        # Compute speed function
        speed_function = compute_speed_function(phi_n, g, alpha=alpha, beta=beta)
        
        # Update φ
        phi_n = phi_n + time_step * speed_function * grad_phi_mag
        
        # Display every 10 iterations
        if ((i+1) % iter_display) == 0 :
            plt.figure(figsize=(20, 4))

            plt.subplot(1, 4, 1)
            plt.imshow(image, cmap='gray')
            plt.contour(phi_n, levels=[0], colors='r')
            plt.title('iteration Contour')
            plt.axis('off')
            
            plt.subplot(1, 4, 2)
            plt.imshow(speed_function, cmap='coolwarm', vmin=-1, vmax=1)
            plt.colorbar()
            plt.title('Speed Function')
            plt.axis('off')

            plt.subplot(1, 4, 3)
            plt.imshow(grad_phi_mag, cmap='coolwarm', vmin=0, vmax=2)
            plt.colorbar()
            plt.title('grad_phi_mag')
            plt.axis('off')
            
            plt.subplot(1, 4, 4)
            plt.imshow(phi_n, cmap='coolwarm', vmin=-1, vmax=1)
            plt.colorbar()
            plt.title('Level Set Function')
            plt.axis('off')
            
            plt.suptitle(f"Iteration {i+1}")
            plt.tight_layout()
            plt.show()

        if (i+1) % reinit_every == 0:
            phi_n = reinitialize_phi(phi_n,  iterations=initialisation_iter, dt=initialisation_time_step)
            phi_x, phi_y, grad_phi_mag_display = compute_phi_gradient(phi_n)
            speed_function_display = compute_speed_function(phi_n, g, alpha=alpha, beta=beta)

            plt.figure(figsize=(20, 4))

            plt.subplot(1, 4, 1)
            plt.imshow(image, cmap='gray')
            plt.contour(phi_n, levels=[0], colors='r')
            plt.title('iteration Contour')
            plt.axis('off')
            
            plt.subplot(1, 4, 2)
            plt.imshow(speed_function_display, cmap='coolwarm', vmin=-1, vmax=1)
            plt.colorbar()
            plt.title('Speed Function')
            plt.axis('off')

            plt.subplot(1, 4, 3)
            plt.imshow(grad_phi_mag_display, cmap='coolwarm', vmin=0, vmax=2)
            plt.colorbar()
            plt.title('grad_phi_mag')
            plt.axis('off')
            
            plt.subplot(1, 4, 4)
            plt.imshow(phi_n, cmap='coolwarm', vmin=-1, vmax=1)
            plt.colorbar()
            plt.title('Level Set Function')
            plt.axis('off')
            
            plt.suptitle(f"Reinitialization : Iteration {i+1}")
            plt.tight_layout()
            plt.show()
    
    # Final display
    plt.figure(figsize=(20, 4))

    plt.subplot(1, 4, 1)
    plt.imshow(image, cmap='gray')
    plt.contour(phi_n, levels=[0], colors='r')
    plt.title('iteration Contour')
    plt.axis('off')
    
    plt.subplot(1, 4, 2)
    plt.imshow(speed_function, cmap='coolwarm', vmin=-1, vmax=1)
    plt.colorbar()
    plt.title('Speed Function')
    plt.axis('off')

    plt.subplot(1, 4, 3)
    plt.imshow(grad_phi_mag, cmap='coolwarm', vmin=0, vmax=2)
    plt.colorbar()
    plt.title('grad_phi_mag')
    plt.axis('off')
    
    plt.subplot(1, 4, 4)
    plt.imshow(phi_n, cmap='coolwarm', vmin=-1, vmax=1)
    plt.colorbar()
    plt.title('Level Set Function')
    plt.axis('off')
    
    plt.suptitle("Final Display")
    plt.tight_layout()
    plt.show()
    
    return phi_n

In [ ]:
# test of the level set with reinitialization algorithm time_step=0.1, alpha=0.1, beta=1.2, initialisation_iter=10, initialisation_time_step=0.1 :

img = cv2.imread("coins.png", cv2.IMREAD_GRAYSCALE)
phi_n = level_set_contour_detection_fundamental_reinitialization(img, 4000, center_initial=None, radius_initial=None, time_step=0.1, alpha=0.1, beta=1.2, reinit_every = None, initialisation_iter=10, initialisation_time_step=0.1)

## Conclusion:

- The LSF transforms into an accurate and regular distance function, that also makes the irregular unwanted contours disapear.
- With the reinitialization we see that the irregular unwanted contours start to dissapear right after the reinitialization.
- This reinitialization process is computationally expensive and can cause the zero level set to shift unintentionally.

# 3 - Variational Level Set Methods (Early 2000s)

A major advancement came with variational formulations where level set evolution was derived by minimizing energy functionals:


$$ E(\phi) = \int\int [\text{internal energy terms} + \text{external energy terms}] , dxdy $$

Where:
- **E_internal**: Controls the regularity of the contour (smoothness, length)
- **E_external**: Drives the contour toward desired features (edges, regions)


The evolution equation became:
$$ ∂φ/∂t = -∂E/∂φ $$


### Common internal energy terms include:

1. **Length regularization**:
   
   $$ E_{\text{length}}(\phi) = \int\int \delta(\phi) |\nabla \phi| , dxdy $$
   
   The term Elength​(ϕ) approximates the length (or perimeter in 2D) of the contour defined by ϕ=0.  
   This penalizes the length of the contour, encouraging smoothness.

    **Dirac Delta Function δ(ϕ)** :
    The function δ(ϕ) is a Dirac delta function that is non-zero only where ϕ=0, i.e., on the contour. It acts like a "selector" that focuses the integral on the zero level set. In practice, δ(ϕ) is approximated by a smooth function (e.g., a Gaussian-like function) to handle numerical computations.

2. **Area constraint**:
   
   $$ E_{\text{area}}(\phi) = \int\int H(-\phi) , dxdy $$
   
   This term controls the area enclosed by the contour, preventing collapse or excessive expansion.

   **Heaviside Function H(−ϕ)** : corresponds to 1 inside of the contour and in the contour, and 0 outside of it.

### External Energy Terms

These are application-specific and draw the contour toward features of interest:

1. **Edge-based terms**:
   
   $$ E_{\text{edge}}(\phi) = \int\int g * \delta(\phi) |\nabla \phi| , dxdy $$
   
   Where g is the edge indicator function that approaches zero at edges.

   The term E_edge​ is a weighted length of the contour, where the weighting is provided by the edge indicator **g**. Since **g** is small at edges, minimizing E_edge​ encourages the contour to align with image edges, where the energy contribution is low. Mathematically:  

    - The δ(ϕ) restricts the integral to the contour (ϕ=0).  
    - The term  ∣∇ϕ∣dxdy contributes to the arc length along the contour.  
    - The factor **g** weights the arc length, making it “cheaper” (lower energy) for the contour to lie on edges where g≈0.  

2. **Region-based terms**:
   
   $$ E_{\text{region}}(\phi) = \int\int (I - c_1)^2 H(-\phi) , dx + \int (I - c_2)^2 (1 - H(-\phi)) , dxdy $$

   This models the image as consisting of regions with different statistical properties (like the Chan-Vese model).

## The Algorithm for Contour Evolution

### Step 1: Formulate the Energy Functional
Define E(φ) with appropriate terms for your application.

### Step 2: Derive the Evolution Equation
Using calculus of variations, we compute the Euler-Lagrange equation:

$$∂φ/∂t = -∂E/∂φ$$


This gradient descent approach ensures φ evolves in the direction that decreases E most rapidly.

### Step 3: Discretize the Evolution Equation
Convert the continuous equation to a discrete update scheme:

$$ \phi^{n+1} = \phi^n + \Delta t \cdot \left(-\frac{\partial E}{\partial \phi}\right) $$


### Step 4: Numerically Implement the Update Scheme

For example, the gradient descent for a simple model might look like:

$$ \phi^{n+1} = \phi^n + \Delta t \cdot \left[ \lambda \delta(\phi) \text{div}\left(\frac{\nabla \phi}{|\nabla \phi|}\right) + \nu \delta(\phi) + \mu \delta(\phi) (I - c_1)^2 - \mu \delta(\phi) (I - c_2)^2 \right] $$


Where:
- First term: Curvature regularization
- Second term: Balloon force
- Third & fourth terms: Region forces

### Step 5: Maintain Numerical Stability
Implement upwind schemes, enforce Courant-Friedrichs-Lewy (CFL) condition, and use regularization.

### Step 6: Handle the Signed Distance Property
In traditional variational methods, this is done through periodic reinitialization.


This provided a more principled approach but still required reinitialization.


# 4 - Development of DRLSE (Li et al., 2010)

Li and colleagues recognized that reinitialization was:
1. Computationally expensive
2. Theoretically artificial (not derived from the original variational problem)
3. Potentially disruptive to the evolving contour

Their key insight was to incorporate distance regularization directly into the energy functional:

$$E(\phi) = \mu R_p(\phi) + \lambda L_g(\phi) + \alpha A_g(\phi)$$

Where:
- $R_p(\phi)$ is the distance regularization term
- $L_g(\phi)$ is the length term
- $A_g(\phi)$ is the area term

### The Distance Regularization Term

The critical innovation was the distance regularization term:

$$R_p(\phi) = \int\int p(|\nabla\phi|) \, dxdy$$

With potential function $p$ designed so that:
- $|\nabla\phi| = 1$ is a minimum of $p(s)$
- The gradient flow of $R_p$ automatically maintains the signed distance property

### Potential Function Design

Li et al. proposed two potential functions:

1. **Single-well**: $p_1(s) = \frac{1}{2}(s-1)^2$
   - Simpler but only works well for region-based models
   
2. **Double-well**: $p_2(s) = \begin{cases}
     \frac{\sin(2\pi s)}{2\pi} & \text{if } s \leq 1 \\
     (s-1) & \text{if } s > 1
   \end{cases}$
   - More sophisticated with better numerical properties
   - Works for both edge-based and region-based models
   - Creates an attractive force toward $|\nabla\phi| = 1$

## Mathematical Connection Between Basic Level Set and DRLSE

The derivation path can be seen as:

1. **Basic level set equation**:
   
   $$\frac{\partial\phi}{\partial t} + F|\nabla\phi| = 0$$

2. **Variational formulation**:
   
   $$\frac{\partial\phi}{\partial t} = -\frac{\partial E}{\partial\phi}$$
   
   Where $E$ typically includes length and area terms:
   
   $$E(\phi) = \lambda\int\delta(\phi)|\nabla\phi|dx + \alpha\int H(-\phi)dx$$

3. **Addition of reinitialization as a separate process**:
   
   $$\frac{\partial\psi}{\partial\tau} = \text{sign}(\phi)(1 - |\nabla\psi|)$$

4. **DRLSE's unification** - incorporating regularization directly:
   
   $$\frac{\partial\phi}{\partial t} = \mu\text{div}(d_p(|\nabla\phi|)\nabla\phi) + \lambda\delta(\phi)\text{div}(g\frac{\nabla\phi}{|\nabla\phi|}) + \alpha g\delta(\phi)$$
   
   Where the first term automatically maintains $|\nabla\phi| \approx 1$ without separate reinitialization.

## Key Advancements of DRLSE

1. **Mathematical elegance**: The complete level set evolution is derived from a single energy functional
2. **Computational efficiency**: Eliminates the need for separate reinitialization
3. **Numerical stability**: Maintains the signed distance property throughout evolution
4. **Contour fidelity**: Prevents artificial movement of the zero level set
5. **Framework flexibility**: Compatible with both edge-based and region-based segmentation

The evolution from basic level set methods to DRLSE represents a significant theoretical advancement that addresses the fundamental issues in level set approaches while maintaining their powerful capabilities for handling topological changes during contour evolution.